In [ ]:
import pandas as pd
import numpy as np
train=pd.read_csv("data/train.csv",na_values=["-1","-1.0"])
test=pd.read_csv("data/test.csv",na_values=["-1","-1.0"])

print(train.shape,test.shape)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.target.sum()/train.shape[0]

In [ ]:
binary = ['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
          'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_calc_15_bin', 
          'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin']
# 범주형 변수
category = ['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 
            'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 
            'ps_car_10_cat', 'ps_car_11_cat']
# 정수형 변수
integer = ['ps_ind_01', 'ps_ind_03', 'ps_ind_14', 'ps_ind_15', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06', 
           'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 
           'ps_calc_14', 'ps_car_11']
# 소수형 변수
floats = ['ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_car_12', 'ps_car_13',
          'ps_car_14', 'ps_car_15']

# 단일변수 히스토그램

In [ ]:
test["target"]=np.nan
all_data=pd.concat([train,test],axis=0)


In [ ]:
def bar_plot(col,data,hue=None):
    f,ax=plt.subplots(figsize=(10,5))
    sns.countplot(x=col,hue=hue,data=data,alpha=0.5)
    plt.show()
def dist_plot(col,data):
    f,ax=plt.subplots(figsize=(10,5))
    sns.distplot(data[col].dropna(),kde=False,bins=10)
    plt.show()

In [ ]:
for col in binary+category+integer:
    bar_plot(col,train)

In [ ]:
for col in floats:
    dist_plot(col,train)

# 변수간 상관관계

In [ ]:
corr=all_data.corr()
cmap=sns.color_palette("Blues")
f,ax=plt.subplots(figsize=(10,7))
sns.heatmap(corr,cmap=cmap)

In [ ]:
features = ['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 
          'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin',
          'ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 
          'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 
          'ps_car_11_cat', 'ps_ind_01', 'ps_ind_03', 'ps_ind_14', 'ps_ind_15', 'ps_car_11',
          'ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_car_12', 'ps_car_13',
          'ps_car_14', 'ps_car_15']

corr_sub=all_data[features].corr()
f,ax=plt.subplots(figsize=(10,7))
sns.heatmap(corr_sub,cmap=cmap)

- ps_ind_14와 ps_ind_12_bin은 강한 상관관계를 가진다
- **95%이상의 강한 상관관계가 없으므로 뺄 데이터는 없다.**


# 단일 변수 vs 타겟 변수

In [ ]:
def bar_plot_ci(col,data):
    f,ax=plt.subplots(figsize=(10,5))
    sns.barplot(x=col,y="target",data=data)
    plt.show()
    
for col in binary+category+integer:
    bar_plot_ci(col,all_data)

#### 전체 데이터 기준 타겟 변수의 비율은 3.6%이다. 이 점을 감안하며 함께 분석해 보자. 막대 그래프 중간에 그려진 검정색 직선은 95% 신뢰구간을 의미한다.
#### ps_ind_16_bin,ps_ind_06_bin 두변수는 이진 변수의 값에 따라 타겟 변수의 비율이 다르다

In [ ]:
tar=["ps_ind_16_bin","ps_ind_06_bin","ps_ind_11_bin","ps_calc_18_bin"]

In [ ]:
for col in tar:
    bar_plot_ci(col,all_data)

1. ps_ind_16_bin,ps_ind_06_bin 두 변수는 이진 변수 값에 따라 타겟 변수의 비율이 다르다. 통계적 유효성을 충분히 지니고 있는 두 변수는 모델링 관점에서 유용하다. 
2. 반면 ps_ind_11_bin은 평균적으로는 다르게 보이지만 통계적으로 유효성이 없다. 
3. ps_calc_18은 target변수의 값이 큰 차이가 없으므로 유효한 변수가 아니다

1. ps_car_01_cat 변수는 7,8 값이 타겟 비율이 다르다. 따라서 의미가 있다. 1과2는 타겟 비율의 95% 신뢰구간이 넓어 변수로써 예측능력이 월등하지는 않다. 
2. ps_car_11_cat 변수는 104개의 고유값이 0.02~0.08의 큰 범위의 타겟 비율을 보인다. 따라서 의미가 있다.
3. ps_calc_06,ps_calc_07 변수는 특정 값에서의 95% 신뢰구간이 너무 광범위해서 예측 능력은 떨어진다.

In [ ]:
features = ['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_12_bin', 
            'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_ind_04_cat', 'ps_ind_05_cat', 
            'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_06_cat', 
            'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_11_cat', 'ps_ind_01', 
            'ps_ind_03', 'ps_ind_15', 'ps_car_11']

+ 위 값이 단일 변수로써 의미가 있는 변수들이다.

# 훈련데이터 vs 테스트데이터 비교

In [ ]:
all_data["is_tst"]=all_data["target"].isnull()
for col in binary+category+integer:
    bar_plot(col,all_data,"is_tst")

In [ ]:
all_data["target"].isnull().sum(),all_data.shape[0],train.shape[0],test.shape[0]

+ 모든 데이터 들이 트레인 데이터와 테스트 데이터 비율이 유사하다. 타겟이 null인 변수는 test변수이다. hue로 null 설정되었으므로 false는 트레인, true는 테스트이다. 모든 변수들이 1:1.5의 비율을 유지하고 있기 때문이다.

# 탐색적 데이터 분석 요약
+ 모든 데이터는 정수, 소수형 타입이라서 바로 모델을 적용할 수 있다.
+ 57개의 변수는 모두 익명화 되어 있다.
+ 전처리 과정으로 정보력이 아예 없는 상수값 혹은 0.95 이상의 높은 상관관계를 가지는 중복 변수등은 존재하지 않는다.
+ 변수의 예측 능력을 가늠하기 위해서 변수 고유값별 타겟 변수의 비율을 신뢰구간 95%와 함께 분석해 보았다. 의미있는 항목을 선별할 수 있다.
- 훈련 데이터와 테스트 데이터를 비교해 보았는데, 데이터 분포으 차이는 없었다.

# baseline 모델

- baseline 모델 pipeline을 구축하는 방법은 다음과 같다
    - 데이터 전처리, 피처 엔지니어링, 학습 모델 정의, 모델 학습 및 교차 검증 평가, 테스트 데이터 예측

### 1) 데이터 전처리

In [ ]:
train=pd.read_csv("data/train.csv")
y_train=train["target"]
train_id=train.id
del train["target"],train["id"]

test=pd.read_csv("data/test.csv")
test_id=test.id
del test["id"]

### 2) 피처 엔지니어링
+ eda를 통해서 얻은 데이터를 적극 활용한다면 유의미한 파생 변수를 생성할 수 있다.
+ 3가지 기초 피처 엔지니어링을 수행한다.
    - 결측값의 개수를 나타내는 missing 변수
    - 이진변수들의 총합
    - target encoding 파생변수

In [ ]:
train["missing"]=(train==-1).sum(axis=1).astype(float)
test["missing"]=(train==-1).sum(axis=1).astype(float)

bin_features=[c for c in train.columns if 'bin' in c]
train["bin_sum"]=train[bin_features].sum(axis=1)
test["bin_sum"]=test[bin_features].sum(axis=1)

In [ ]:
features = ['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_12_bin', 
            'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_ind_04_cat', 'ps_ind_05_cat', 
            'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_06_cat', 
            'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_11_cat', 'ps_ind_01', 
            'ps_ind_03', 'ps_ind_15', 'ps_car_11']

+ 첫번째 파생 변수는 nan값을 더한 것이다. 독자는 의야해 할 수도 있지만, 과거 경진대회에서 효자로 작용한 적이 종종 있다.
+ 두번째 파생 변수는 이진 변수의 합이다. 변수 간의 상호 작용으로 얻을 수 있는 고차원 정보를 추출한다.
+ 세 번재 파생 변수는 데이터 탐색 분석 과정에서 선별한 일부 변수들을 대상으로 Target encoding을 수행한다.
    + 예를들어 ps_ind_01 변수가 0일 경우 ps_ind_01 변수가 0인 모든 운전자들의 평균 타겟값을 ps_ind_01_enc 파생변수로 생성하는 것이다.
        - 타겟 변수의 값을 직접적으로 사용하는 변수이기 때문에 데이터 유출로 이어져 모델 파이프라인을 망칠 수 있다
        - 데이터 유출을 방지하기 위해서, 5-fold 내부 교차 검증 과정에서 학습에 사용되는 4/5 훈련 데이터로 변수 고유값별 평균 타겟 계산
        - 1/5의 검증 데이터에 해당 값을 매핑하는 방식을 취한다.

### 3) LightBGM 모델 정의
+ num_leaves, max_bin,min_child_samples 설정으로 모델 복잡도를조절
+ feature_fraction, subsample,max_drop 설정값으로 과적합을 방지


In [ ]:
import lightgbm as lgbm
num_boost_round = 10000
params = {"objective": "binary",
          "boosting_type": "gbdt",
          "learning_rate": 0.1,
          "num_leaves": 15,
          "max_bin": 256,
          "feature_fraction": 0.6,
          "verbosity": 0,
          "drop_rate": 0.1,
          "is_unbalance": False,
          "max_drop": 50,
          "min_child_samples": 10,
          "min_child_weight": 150,
          "min_split_gain": 0,
          "subsample": 0.9,
          "seed": 2018
}

### 4) 모델 학습 및 교차 검증 평가
+ 교차검증은 5-fold StratifiedKFold 기법을 사용한다.
+ 시계열 데이터가 아니므로 제공된 데이터를 랜덤하게 분리하여 교차 검증
+ 재현성을 위하여 random_state를 고정한다.

In [ ]:
from sklearn.model_selection import StratifiedKFold
NFOLDS=5
kfold=StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=218)
kf=kfold.split(train,y_train)

In [ ]:
cv_train=np.zeros(len(y_train))
cv_pred=np.zeros(len(test_id))
best_trees=[]
fold_scores=[]

In [ ]:
kfold,kf

In [ ]:
def Gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]

    # sort rows on prediction column
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:, 0].argsort()][::-1, 0]
    pred_order = arr[arr[:, 1].argsort()][::-1, 0]

    # get Lorenz curves
    L_true = np.cumsum(true_order) * 1. / np.sum(true_order)
    L_pred = np.cumsum(pred_order) * 1. / np.sum(pred_order)
    L_ones = np.linspace(1 / n_samples, 1, n_samples)

    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)

    # normalize to true Gini coefficient
    return G_pred * 1. / G_true

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', Gini(labels, preds), True

In [ ]:
for i,(train_fold,validate) in enumerate(kf):
    x_train,x_validate,label_train,label_validate=\
    train.iloc[train_fold,:],train.iloc[validate,:],y_train[train_fold],y_train[validate]
    
    for feature in features:
        map_dic=pd.DataFrame([x_train[feature],label_train]).T.groupby(feature).agg('mean')
        map_dic=map_dic.to_dict()["target"]
        x_train[feature+"_target_enc"]=x_train[feature].apply(lambda x:map_dic.get(x,0))
        x_validate[feature+"_target_enc"]=x_validate[feature].apply(lambda x : map_dic.get(x,0))
        test[feature+"_target_end"]=test[feature].apply(lambda x:map_dic.get(x,0))

    dtrain=lgbm.Dataset(x_train,label_train)
    dvalid=lgbm.Dataset(x_validate,label_validate,reference=dtrain)

    bst=lgbm.train(params,dtrain,num_boost_round,valid_sets=dvalid,feval=evalerror,verbose_eval=100,early_stopping_rounds=100)
    best_trees.append(bst.best_iteration)
    cv_pred+= bst.predict(test,num_iteration=bst.best_iteration)
    cv_train[validate] += bst.predict(x_validate)
    
    score=Gini(label_validate,cv_train[validate])
    print(score)
    fold_scores.append(score)


In [ ]:
cv_pred/=NFOLDS

print("cv score:")
print(Gini(y_train,cv_train))
print(fold_scores)
print(best_trees,np.mean(best_trees))

pd.DataFrame({"id":test_id,"target":cv_pred}).to_csv("Data/lgbm_baseline.csv",index=False)